## Tool to select random <a href="http://tectonique.net/noddy">Noddy</a> models from database
<br>First load libraries

In [ ]:
import  psycopg2
import numpy as np
import matplotlib
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import functools 
import operator  
from random import seed
import random
import time
import binascii
%matplotlib inline

exec(open("connectomatic.py").read())


Now connect to database and selct some models based on <b>event_filter</b> to subset possible event histories. We will save a temporary copy of the history files for each mode, but first we empty the directory where they will be stored.

In [ ]:
connection=connectomatic()
modelrangemin=1
show=1
savehis=1
event_filter="STRAT-TILT-%-%-FOLD"

if(savehis):
    import os
    import glob
    files = glob.glob('his/*')
    for f in files:
        os.remove(f)



Now get the models and display the mag, grav and map views

In [ ]:
now = time.time()
# seed random number generator
seed(int(now))

allmodels=getSome("COUNT(*)", 'public.universe', '', 0, '', 0)
data=getSome("COUNT(*)", 'public.universe', 'events', 0, event_filter, 0)

print("                                     Models ",modelrangemin,"-",data-2," of ",allmodels)
print("                                 Using filter = ",event_filter)
count=data

for z in range (1,9):
    
    EVENTIDS=getSome("ID", 'public.universe', 'events', 0, event_filter, 1)
    ran =random.randint(modelrangemin,data-2) 
    IDX=EVENTIDS[ran]
    if(IDX>99900 and IDX < 112565):
        continue
    #IDX=allmodels
    
    IDX=str(IDX)

    UID=getSome("UID", 'public.universe', 'ID', 1, IDX, 0)
    
    legend=getSome("events", 'public.universe', 'ID', 1, IDX, 0)
    legend=legend.replace("-","  |  ")    
    
    strx=getSome("his", 'public.history', 'UID', 1, UID, 0)
    
    if(savehis):
        f= open('his/'+IDX+'.his',"w+")
        f.write(strx.replace('\t',''))
        f.close()
    
    strx=getSome("grv", 'public.gravity', 'UID', 1, UID, 0)

    imj = np.ones((200,200))
    pix=strx.strip().split(' ')
    #np.float_(pix)
    #imj9 =np.ones((4000))
    #imj9 = np.asarray(pix) 
    #imj9=np.reshape(imj9,(200,200))
    strx=strx.replace("\n"," ")

    k=0
    for i in range (0, 200, 1):
        m=0
        for j in range (0, 200, 1):
            imj[i][m]=pix[k]
            k+=1
            m+=1

    strx=getSome("mag", 'public.magnetics', 'UID', 1, UID, 0)

    imj2 = np.ones((201,201))

    strx=strx.replace("  "," ")
    strx=strx.replace("\n"," ")
    
    pix=strx.split(' ')

    k=0
    for i in range (0, 200, 1):
        m=0
        for j in range (0, 200, 1):
            if(pix[k]!=""):
                imj2[i][m]=float(pix[k])
            k+=1
            m+=1

    strx=getSome("g12", 'public.BLOCK', 'UID', 1, UID, 0)
    
    block = np.ones((201,201,201))
    imj3 = np.ones((201,201))
    
    i=0
    for j in strx:
        c=i%200
        r=int((i-c)/200)
        if(r>199):
            break        
        imj3[199-r][c]=float(ord(j))        
        i+=1
    
    maxcol=np.amin(imj3)
    for i in range (0,199):
        for j in range (0,199):
            imj3[i][j]=imj3[i][j]*maxcol
            
    fig, ax = plt.subplots(1,3)
    fig.tight_layout()
    if(show==1):                     
        ax[0].title.set_text(f'                                                               {IDX:6} {legend} \nMag' )
        ax[1].title.set_text('Grav')
        ax[2].title.set_text('Map')
        ax[0].imshow(imj2,cmap='rainbow');
        ax[1].imshow(imj,cmap='rainbow');
        ax[2].imshow(imj3,cmap='tab20c');
        plt.show()
    else:
        matplotlib.image.imsave('grv/model_'+UID+'_grv.png', imj,cmap='rainbow')
        matplotlib.image.imsave('mag/model_'+UID+'_mag.png', imj2,cmap='rainbow')
        matplotlib.image.imsave('map/model_'+UID+'_map.png', imj3,cmap='tab20')
        
    print("             http://localhost:8888/edit/Dropbox/1_Jupyter_notebooks/noddyverse/his/"+IDX+".his")